In [1]:
import re
import pandas as pd
from urllib2 import urlopen
import json
import requests
from lxml import html
import codecs

def get_jsonparsed_data(url):
    response = urlopen(url)
    data = str(response.read())
    return json.loads(data)

def parseGameCode(CD):
    S = str(int(CD))
    return [int(S[0:4]), int(S[4:6]), int(S[-4:])]

def getURL(seas, gametype, gamenum, repcode):
    url = ['http://www.nhl.com/scores/htmlreports/', str(seas - 1), str(seas),
           '/', repcode, '0', str(gametype), ('%04i' % (gamenum)), '.HTM']
    return ''.join(url)

def getReport(gameCode, ReportCode):
    GC = parseGameCode(gameCode)
    url = getURL(GC[0], GC[1], GC[2], ReportCode)
    req = requests.get(url, headers={
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'
    })
    return req

def SaveEvt(f, gameId, per, playNo, event, desc, time, stg):
    T = str(int(gameId)) + ',' + str(per) + ',' + str(
        playNo) + ',\"' + event + '\",\"' + desc + '\",' + time + ',' + stg
    file = codecs.open(f, 'a', 'utf-8')
    file.write(T + '\n')
    file.close()

def SaveEvtRO(f, gameId, per, playNo, ha, plNo, plNa, PlPo):
    T = str(int(gameId)) + ',' + str(per) + ',' + str(playNo) + ',' + ha + ',' + plNo + ',' + plNa + ',' + PlPo
    file = codecs.open(f, 'a', 'utf-8')
    file.write(T + '\n')
    file.close()

NHLTeams = ['ANA','ARI','BOS','BUF','CAR','CBJ','CGY','CHI','COL','DAL','DET','EDM','FLA','LAK','MIN','MTL','NJD',\
            'NSH','NYI','NYR','OTT','PHI','PHX','PIT','SJS','STL','TBL','TOR','VAN','WPG','WSH']
 
Years_to_Scrape = [2013, 2014, 2015,2016]
ColMap = {'play_num': 0, 'per': 1, 'str': 2, 'time': 3, 'event': 4, 'desc': 5, 'vis': 6, 'home': 7}
#Schedule per Month http://nhlwc.cdnak.neulion.com/fs1/nhl/league/clubschedule/CGY/2014/02/iphone/clubschedule.json

In [3]:
# GET SCHEDULE
Schedule = pd.DataFrame()
for TeamID in NHLTeams:
    for Yr in Years_to_Scrape:
        for Mo in range(1,13):
            try:
                LinkID = 'http://nhlwc.cdnak.neulion.com/fs1/nhl/league/clubschedule/' + TeamID + '/' + str(Yr) + '/' + str(Mo) + '/iphone/clubschedule.json'
                LinkData = get_jsonparsed_data(LinkID)
                GameData = pd.DataFrame(LinkData['games'])
                GameData['Team'] = TeamID
                GameData['Yr'] = Yr
                GameData['Mo'] = Mo
                Schedule = pd.concat([Schedule, GameData], axis=0)
            except:
                ErrStr = TeamID + '-' + str(Yr) + '-' + str(Mo)

In [4]:
import codecs
fileName = 'NewGameInfo.csv'
ERecs = pd.read_csv(fileName, encoding = 'utf-8')
ExistingRecs = ERecs.gameId
file = codecs.open(fileName,'a','utf-8')
#Hdr = 'gameId,home,visitor,att,date,loc'
#file.write(Hdr + '\n')

In [5]:
for H, G in Schedule.iterrows():
    if G.gameId not in ExistingRecs:
        GameId = G.gameId
        HTML_Obj = getReport(GameId, 'PL')
        hTree = html.fromstring(HTML_Obj.content)
        try:
            vFull = team_scr(hTree,'Visitor')[1]
            hFull = team_scr(hTree, 'Home')[1]

            game_info = hTree.xpath('//table[@id="GameInfo"]')[0].xpath('.//text()')
            game_info = '; '.join(s.strip() for s in game_info if s.strip() != '')

            att = re.findall(r'(?<=[aA]ttendance\s)(\d*\,?\d*)', game_info)
            att = int(att[0].replace(',','')) if att else 0

            date = re.findall(r'\w+\,?\s\w+\s\d+\,?\s\d+', game_info)
            date = date[0] if date else ''
            date = date.replace(',','')
            loc = re.findall(r'(?<=at\W)([^\;]*)', game_info)
            loc = loc[0] if loc else ''

            rec = str(int(GameId)) + ',' + vFull + ',' + hFull + ',' + str(att) + ',' + date + ',' + loc
            file.write(rec + '\n')
        except:
            pass
file.close()

<Element html at 0xb0e7e08>